In [18]:
import imp
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
from torchvision import transforms
from networks import *
from tqdm import tqdm
import wandb
import numpy as np
from embeddings_utils import *

In [5]:
#Get Resnet pre-trained model to compute embeddings of MIT_split
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

print(list(model.children())[-1])
newModel = torch.nn.Sequential(*(list(model.children())[:-1]))
print(list(newModel.children())[-1])



Using cache found in C:\Users\galve/.cache\torch\hub\pytorch_vision_v0.10.0


Linear(in_features=512, out_features=1000, bias=True)
AdaptiveAvgPool2d(output_size=(1, 1))


In [11]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5],
    #                      std=[0.5, 0.5, 0.5] ) 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #For imagenet pretrained
])
TRAIN_DATA_PATH = "../MIT_split/train/"
